In [5]:
import os
import string
import re
import math
import statistics
from math import sqrt
import numpy as np
import random
import time
import pandas as pd
import nltk, re, pprint
#nltk.download('punkt')
from nltk import word_tokenize
random.seed(1)
np.random.seed(1)
import matplotlib.pyplot as plt
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

import pickle
import datetime

import gensim
from gensim.models import word2vec

#os.chdir('C:/Users/torna/Documents/StatApp/StatApp')
#os.chdir('/Users/alainquartierlatente/Desktop/Ensae/StatApp')
#os.chdir('/home/aqlt/Documents/Ensae/StatApp')
os.chdir('C:/Users/Kim Antunez/Documents/Projets_autres/StatApp')

#nom_dossier = "ens"
nom_dossier = "100k"

In [7]:
# Penser à changer selon taille
with open("data/corpus_trie%s.file" %nom_dossier, "rb") as f:
    corpus = pickle.load(f) 
ens_tweets = [phrase.split() for phrase in corpus]
phrases = ens_tweets
print(len(phrases))

99718


# Lancement du modèle

### Nouvelle simulation 

Paramètres fixés : le nombre de mots tirés dans le *negative sampling* et la proba utilisé

Paramètres à modifier :

In [8]:
seeds = [1, 5, 10, 15, 20, 25]

# renseigner dans l'ordre : dim, epoch, window, learning_rate
#simuls = [(100,100,4,0.02),(50,10,3,0.01)]
simuls = [(100,80,4,0.02)]
#dim = 100 / ep = 100 / w = 4 / lr = 0.02

# Rq : numero_simulation n'est plus utilisé désormais ! 

In [9]:
for dim, epoch, window, learning_rate in simuls: #suppr

    for seed in seeds:

        start = time.time()

        model = word2vec.Word2Vec(phrases, size=dim, window=window,negative=5,
                                  alpha=learning_rate,min_alpha=learning_rate, seed=seed,
                          min_count=0, workers=4, iter=epoch)
        # Kim : j'ai mis min_count = 0, devons-nous mettre 4 ?
        
        end = time.time()
        # Affichage temps de tournage du modèle
        currentDT = datetime.datetime.now()
        print("seed {} à {}:{} : {}".format(seed, currentDT.hour, currentDT.minute, round((end - start)/60, 2)))

        # Sauvegarde 
        chemin = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed{}".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:], seed)
        if not os.path.isdir(chemin):
            os.mkdir(chemin)

        model.save(chemin + "/word2vec.model")


# phrases = les phrases du corpus
# size = dimension du vecteur
# window = fenetre
# negative = nb de neg samples
# alpha = learning rate
# seed 
# min_count = fréquence min des mots qui nous intéresse
# workers = nb de cores sur l'ordi
# iter = nb epoch

seed 1 à 1:36 : 1.15
seed 5 à 1:38 : 1.26
seed 10 à 1:39 : 1.29
seed 15 à 1:40 : 1.27
seed 20 à 1:41 : 1.04
seed 25 à 1:42 : 1.08


### Simulation déjà effectuée (faire de nouvelles epoch)

CETTE PARTIE BUG ET EST A RECODER !

Paramètres à modifier :

In [5]:
epoch = 1 #Nouveau nombre d'epoch à effectuer
learning_rate = 0.01
numero_simulation = 100
seed = 1
version = 1 #version du fichier à modifier : pex version = "1" pour word2vec1.model

In [ ]:
chemin = "data/" + nom_dossier + "/gensim/Simulation_" + str(numero_simulation) + "_seed" + str(seed)       
model = gensim.models.keyedvectors.KeyedVectors.load(chemin + "/word2vec" + str(version) +".model")  

start = time.time()

#### NE MARCHE PAS ! 
#model.train(phrases, epochs=epoch, total_examples=model.corpus_count, 
#            start_alpha=learning_rate, end_alpha=learning_rate) #, compute_loss=True
   
end = time.time()

# Sauvegarde 
#model.save(chemin + "/word2vec" + str(version+1) +".model")

Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\Kim Antunez\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\Kim Antunez\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Kim Antunez\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py", line 211, in _worker_loop
    tally, raw_tally = self._do_train_job(data_iterable, job_parameters, thread_private_mem)
  File "C:\Users\Kim Antunez\Anaconda3\lib\site-packages\gensim\models\word2vec.py", line 821, in _do_train_job
    tally += train_batch_cbow(self, sentences, alpha, work, neu1, self.compute_loss)
  File "gensim/models/word2vec_inner.pyx", line 622, in gensim.models.word2vec_inner.train_batch_cbow
  File "gensim/models/word2vec_inner.pyx", line 489, in gensim.models.word2vec_inner.init_w2v_config
TypeError: object of type 'NoneType' has no len()

Exception in thread Thread-8:
Traceback (most recent

#  Quelques fonctions intégrées dans gensim (non utilisées)

In [9]:
model.corpus_count

99718

In [10]:
vocab = model.wv.vocab
list(vocab)[:5]

['il', 'mérite', 'd', 'être', 'bloquer']

In [11]:
model.wv['wesh']

array([-0.8029589 ,  0.5978353 , -0.49241248, -0.5030643 , -0.03345106,
       -0.0076034 ,  0.26484555,  0.8762143 , -0.4996161 ,  0.08204883,
       -0.08663379,  0.75073415,  0.6436939 , -0.14495811,  0.647974  ,
        0.6855672 , -0.269488  ,  0.2696291 ,  0.49807718, -0.59831595],
      dtype=float32)

In [12]:
model.similarity("homme","femme")

C:\Users\Kim Antunez\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.8841527

Biblio : 
    
https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
http://kavita-ganesan.com/how-to-incorporate-phrases-into-word2vec-a-text-mining-approach/#.XmvT7XIiE2x
https://rare-technologies.com/word2vec-tutorial/
https://radimrehurek.com/gensim/models/word2vec.html